# District migration in Slovakia
The aim of this notebook is to generate graphs for inter-district migration is Slovak districts

In [99]:
import os
import pandas as pd
import geopandas
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Markdown

### Net migration map
- Creates a map of districts and their population change

In [84]:
net_migration_raw = pd.read_csv("../../Data/slovakia/net_migration_districts_sk.csv")

districts_geodata = geopandas.read_file("../../Data/geography/slovak_districts.json")

# display(net_migration_raw.head())
# display(districts_geodata.head())

districts_net_migration = pd.merge(districts_geodata,net_migration_raw,on="LAU1_CODE")

display(districts_net_migration.head())

,LAU1,DOW,IDN3,NM3,IDN2,NM2,NUTS2,NUTS2_CODE,NUTS3,NUTS3_CODE,...,2003,2002,2001,2000,1999,1998,1997,1996,1995,1993
0,Banská Bystrica,2021-02-26 00:00:00+00:00,601,Banská Bystrica,6,Banskobystrický,Stredné Slovensko,SK03,Banskobystrický kraj,SK032,...,-109,-14,67,4,11,-166,-19,149,242,6
1,Banská Štiavnica,2021-02-26 00:00:00+00:00,602,Banská Štiavnica,6,Banskobystrický,Stredné Slovensko,SK03,Banskobystrický kraj,SK032,...,10,39,26,15,46,97,100,-18,22,-7
2,Bardejov,2021-02-26 00:00:00+00:00,701,Bardejov,7,Prešovský,Východné Slovensko,SK04,Prešovský kraj,SK041,...,-93,-85,-79,-96,-63,-79,-52,-12,-47,75
3,Bratislava I,2021-02-26 00:00:00+00:00,101,Bratislava I,1,Bratislavský,Bratislavský kraj,SK01,Bratislavský kraj,SK010,...,-329,-238,-169,-43,-157,-240,-169,126,271,-3
4,Bratislava II,2021-02-26 00:00:00+00:00,102,Bratislava II,1,Bratislavský,Bratislavský kraj,SK01,Bratislavský kraj,SK010,...,340,219,218,157,-150,-35,55,226,128,293


#### Largest/smallest value in counties

In [138]:
def district_value_total(table,ascending = True):
    data = []
    
    for index,row in table.iterrows():
        name = row["NM3"]
        geography = row["geometry"]
        total = 0
        for year in range(1995,2024,1):
            total += row[str(year)]

        data.append({"District":name,
                     "geometry":geography,
                     "value":total})
    result_df = pd.DataFrame(data)
    result_df.sort_values(by="value",ascending=ascending,inplace=True)
    # display(result_df.head())
    return result_df

# district_value_total(districts_net_migration,ascending=False)

#### Creating drawing function

In [ ]:
from matplotlib.colors import Normalize, LinearSegmentedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable

def draw_districts(table, target_column, color_scheme="rgb", title_label="", colorbar_label="Value",fixed_max_value=None,filename="False"):
    if fixed_max_value is None:
        max_value = max([abs(x) for x in table[target_column]])
        # print("Max value: ",max_value)
    else:
        max_value = fixed_max_value
    cmap = LinearSegmentedColormap.from_list("red_green", ["red", "lightgray", "green"])
    norm = Normalize(vmin=-max_value, vmax=max_value)

    # axes = table.plot(column = target_column,legend=True,cmap=cmap,norm=norm,edgecolor="gray",linewidth=0.3)

    fig, ax = plt.subplots(figsize=(8, 6))
    table.plot(
        column=target_column,
        cmap=cmap,
        norm=norm,
        edgecolor="gray",
        linewidth=0.3,
        ax=ax,
        legend=False
    )

    # I used ChatGPT to write this paragraph of code
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="3%", pad=0.1)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm._A = []
    cbar = plt.colorbar(sm, cax=cax)
    cbar.ax.tick_params(labelsize=8)
    cbar.set_label(colorbar_label, fontsize=10)


    ax.set_title(title_label, fontsize=18,  pad=15)
    ax.set_axis_off()
    for spine in ax.spines.values():
        spine.set_visible(True)
        spine.set_edgecolor('black')
        spine.set_linewidth(1)

    #Net gain and loss
    gain_table = districts_net_migration.sort_values(str(target_column),ascending=False).head()
    loss_table = districts_net_migration.sort_values(str(target_column),ascending=False).tail()

    #constant, change here:
    n_districts = 5
    gain = []
    loss = []
    
    for i in range(n_districts):
        gain.append((gain_table.iloc[i,0],int(gain_table.iloc[i,:][str(target_column)])))
        loss.append((loss_table.iloc[i,0],int(loss_table.iloc[i,:][str(target_column)])))
    
    gain_text = r"$\bf{Biggest\ net\ gain:}$"+"\n"
    for district,value in gain:
        gain_text += district + ": "+ str(value) + "\n"
    
    loss_text = r"$\bf{Biggest\ net\ loss:}$"+"\n"
    for district,value in loss:
        loss_text += district + ": "+ str(value) + "\n"

    plt.figtext(0.4, 0.1, gain_text, ha='left', fontsize=9, wrap=False)
    plt.figtext(0.6, 0.1, loss_text, ha='left', fontsize=9, wrap=False)

    plt.tight_layout()
    if filename != "":
        # os.makedirs("../../Images/Slovakia",exist_ok=True)
        plt.savefig("../../Images/Slovakia/"+filename+".png")
        plt.close()
    else:
        plt.show()

# draw_districts(districts_net_migration, str(2020), title_label="District migration in "+str(2020), colorbar_label="Total migration change",filename="")


#### Drawing multiple maps

In [ ]:
# for year in range(1995,2024,1):
#     draw_districts(districts_net_migration, str(year), title_label="District migration in "+str(year), colorbar_label="Total migration change",fixed_max_value=None,filename="Migration/Relative/district_migration_change_"+str(year))

# for year in range(1995,2024,1):
#     draw_districts(districts_net_migration, str(year), title_label="District migration in "+str(year), colorbar_label="Total migration change",fixed_max_value=3000,filename="Migration/Uniform/uniform_district_migration_change_"+str(year))

# draw_districts(districts_net_migration, str(1993), title_label="District migration in "+str(1993), colorbar_label="Total migration change",fixed_max_value=None,filename="Migration/Relative/district_migration_change_"+str(1993))
# draw_districts(districts_net_migration, str(1993), title_label="District migration in "+str(1993), colorbar_label="Total migration change",fixed_max_value=3000,filename="Migration/Uniform/uniform_district_migration_change_"+str(1993))


# draw_districts(districts_net_migration, str(2020), title_label="District migration in "+str(2020), colorbar_label="Total migration change",filename="")


#### Drawing total change


In [ ]:
#Total change
total_table = district_value_total(districts_net_migration,True)
draw_districts(total_table,"value",title_label="Total district migration from 1995 to 2023")

,District,geometry,value
7,Bratislava V,POLYGON ((-568838.9200000018 -1291406.76999999...,-17699
24,Košice III,"POLYGON ((-257734.5 -1238885.75, -257766 -1238...",-7257
23,Košice II,POLYGON ((-266711.30000000075 -1248792.9899999...,-7187
60,Spišská Nová Ves,POLYGON ((-313920.4600000009 -1226105.60999999...,-5400
17,Humenné,POLYGON ((-209754.12000000104 -1227503.0300000...,-5350


### Most immigration/emigration districts

In [103]:
display(Markdown("### Districts with biggest net gain in 2023"))
display(districts_net_migration.sort_values("2023",ascending=False).head())

display(Markdown("### Districts with biggest net loss in 2023"))
display(districts_net_migration.sort_values("2023",ascending=True).head())


### Districts with biggest net gain in 2023

,LAU1,DOW,IDN3,NM3,IDN2,NM2,NUTS2,NUTS2_CODE,NUTS3,NUTS3_CODE,...,2003,2002,2001,2000,1999,1998,1997,1996,1995,1993
55,Senec,2021-02-26 00:00:00+00:00,108,Senec,1,Bratislavský,Bratislavský kraj,SK01,Bratislavský kraj,SK010,...,789,1002,550,576,268,283,183,125,-32,35
13,Dunajská Streda,2021-02-26 00:00:00+00:00,201,Dunajská Streda,2,Trnavský,Západné Slovensko,SK02,Trnavský kraj,SK021,...,770,498,300,169,253,377,261,146,88,53
21,Košice - okolie,2021-02-26 00:00:00+00:00,806,Košice - okolie,8,Košický,Východné Slovensko,SK04,Košický kraj,SK042,...,600,660,412,304,287,513,333,538,-31,25
5,Bratislava III,2021-02-26 00:00:00+00:00,103,Bratislava III,1,Bratislavský,Bratislavský kraj,SK01,Bratislavský kraj,SK010,...,127,400,519,140,100,-159,177,214,115,201
3,Bratislava I,2021-02-26 00:00:00+00:00,101,Bratislava I,1,Bratislavský,Bratislavský kraj,SK01,Bratislavský kraj,SK010,...,-329,-238,-169,-43,-157,-240,-169,126,271,-3


### Districts with biggest net loss in 2023

,LAU1,DOW,IDN3,NM3,IDN2,NM2,NUTS2,NUTS2_CODE,NUTS3,NUTS3_CODE,...,2003,2002,2001,2000,1999,1998,1997,1996,1995,1993
23,Košice II,2021-02-26 00:00:00+00:00,803,Košice II,8,Košický,Východné Slovensko,SK04,Košický kraj,SK042,...,-383,-502,-12,-53,135,-308,-728,-518,-158,-322
71,Vranov nad Topľou,2021-02-26 00:00:00+00:00,713,Vranov nad Topľou,7,Prešovský,Východné Slovensko,SK04,Prešovský kraj,SK041,...,-84,-127,-114,-27,44,-31,59,-18,-34,-17
22,Košice I,2021-02-26 00:00:00+00:00,802,Košice I,8,Košický,Východné Slovensko,SK04,Košický kraj,SK042,...,20,0,50,105,49,-14,1052,873,541,114
60,Spišská Nová Ves,2021-02-26 00:00:00+00:00,810,Spišská Nová Ves,8,Košický,Východné Slovensko,SK04,Košický kraj,SK042,...,-156,-45,-46,6,-138,-91,-51,-28,13,-88
7,Bratislava V,2021-02-26 00:00:00+00:00,105,Bratislava V,1,Bratislavský,Bratislavský kraj,SK01,Bratislavský kraj,SK010,...,-976,-847,-663,-523,-470,-757,-593,-701,-565,-1097
